In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import requests
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
import fiona
from shapely.geometry import Point

In [ ]:
# from methods import read_shst_extract
# from methods import link_df_to_geojson
# from methods import point_df_to_geojson

In [ ]:
# data_interim_dir = "../../data/interim/"

# Prepare tomtom for conflation

In [2]:
%%time
# loading tomtom data source

# gdb_file = "Z:/Data/Users/Sijia/MTC/tomtom/Network_region.gdb"
gdb_file = 'C:/Users/ywang/Box/Modeling and Surveys/Development/Travel Model Two Development/Model Inputs/TomTom networkFGDB/network2019/Network_region.gdb'

# Get all the layers from the .gdb file
layers = fiona.listlayers(gdb_file)
print(layers)

# street link layer
# tomtom_raw_gdf = gpd.read_file(gdb_file, layer = 'mn_nw_BayArea')
tomtom_raw_gdf = gpd.read_file(gdb_file, layer = 'mn_nw')

# convert to ESPG lat-lon
tomtom_raw_gdf = tomtom_raw_gdf.to_crs({'init': 'epsg:4326'})
print(tomtom_raw_gdf.crs)

['mn_sg', 'mn_cf', 'mn_axas', 'mn_axav', 'mn_sm', 'mn_nw', 'mn_jc', 'mn_ln', 'mn_mn', 'mn_pi', 'mn_rf', 'mn_ts', 'mn_2r', 'mn_ig', 'mn_is', 'mn_ld', 'mn_le', 'mn_lf', 'mn_li', 'mn_ll', 'mn_lt', 'mn_rfnm', 'mn_pm', 'mn_lp', 'mn_mp', 'mn_pc', 'mn_rd', 'mn_rn', 'mn_rs', 'mn_td', 'mn_tg', 'mn_tp', 'mn_pcnm', 'mn_si', 'mn_sp', 'mn_sr', 'mn_st', 'mn_tc', 'mn_tl', 'mn_to', 'mn_tt', 'mn_cn', 'mn_isnm', 'mn_jcea']


C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


+init=epsg:4326 +type=crs
Wall time: 2min 56s


In [3]:
tomtom_raw_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 959969 entries, 0 to 959968
Data columns (total 52 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   ID            959969 non-null  float64 
 1   FEATTYP       959969 non-null  int64   
 2   FT            959969 non-null  int64   
 3   F_JNCTID      959969 non-null  float64 
 4   F_JNCTTYP     959969 non-null  int64   
 5   T_JNCTID      959969 non-null  float64 
 6   T_JNCTTYP     959969 non-null  int64   
 7   PJ            959969 non-null  int64   
 8   METERS        959969 non-null  float64 
 9   FRC           959969 non-null  int64   
 10  NETCLASS      959969 non-null  int64   
 11  NETBCLASS     959969 non-null  int64   
 12  NET2CLASS     959969 non-null  int64   
 13  NAME          959969 non-null  object  
 14  NAMELC        959969 non-null  object  
 15  SOL           959969 non-null  int64   
 16  NAMETYP       959969 non-null  int64   
 17  CHARGE        959969 

In [4]:
# There is no existing unique tomtom handle for Bay Area, thus we need to create unique handle

print('total # tomtom links: {}'.format(tomtom_raw_gdf.shape[0]))
print('unique ID + F + T: {}'.format(len(tomtom_raw_gdf.groupby(["ID", "F_JNCTID", "T_JNCTID"]).count())))

# generating unique handle for tomtom
tomtom_raw_gdf["tomtom_link_id"] = range(1, len(tomtom_raw_gdf)+1)

total # tomtom links: 959969
unique ID + F + T: 959969


In [44]:
# Partition tomtom by sub-region boundaries for shst match

# for i in range(14):
#     boundary_gdf = gpd.read_file("../../data/external/county_boundaries/boundary_"+str(i+1)+".geojson")
    
#     sub_tomtom_gdf = tomtom_raw_gdf[tomtom_raw_gdf.intersects(boundary_gdf.geometry.unary_union)].copy()
    
#     sub_tomtom_gdf[["tomtom_link_id", "geometry"]].to_file("../../data/external/tomtom/tomtom"+str(i+1)+".in.geojson",
#                                                 driver = "GeoJSON")

In [5]:
# export modified raw data for merging the conflation results back
tomtom_raw_gdf.to_file("../../data/external/tomtom/tomtom_raw.geojson", driver = "GeoJSON")

# Prepare TM2 for conflation

In [10]:
%%time
# loading TM2 non Marin version data source
# http://bayareametro.github.io/travel-model-two/input/#roadway-network

tm2_link_file = "../../data/external/TM2_nonMarin/mtc_final_network_base.shp"

# street link layer
tm2_link_gdf = gpd.read_file(tm2_link_file)

# define initial ESPG
tm2_link_gdf.crs = {"init" : "esri:102646"}
# if the above throws an "invalid crs" error, try the following:
# tm2_link_gdf.crs = "esri:102646"

# convert to ESPG lat-lon
tm2_link_gdf = tm2_link_gdf.to_crs({'init': 'epsg:4326'})
print(tm2_link_gdf.crs)

+init=epsg:4326 +type=crs
Wall time: 41.2 s


In [11]:
tm2_link_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1405145 entries, 0 to 1405144
Data columns (total 41 columns):
 #   Column       Non-Null Count    Dtype   
---  ------       --------------    -----   
 0   A            1405145 non-null  int64   
 1   B            1405145 non-null  int64   
 2   NUMLANES     1405145 non-null  int64   
 3   F_JNCTID     1405145 non-null  int64   
 4   T_JNCTID     1405145 non-null  int64   
 5   FRC          1405145 non-null  int64   
 6   NAME         741995 non-null   object  
 7   FREEWAY      1405145 non-null  int64   
 8   TOLLRD       291 non-null      object  
 9   ONEWAY       129292 non-null   object  
 10  KPH          1405145 non-null  int64   
 11  MINUTES      1405145 non-null  float64 
 12  CARRIAGE     100 non-null      object  
 13  LANES        1405145 non-null  int64   
 14  RAMP         1405145 non-null  int64   
 15  SPEEDCAT     1405145 non-null  int64   
 16  FEET         1405145 non-null  float64 
 17  ASSIGNABLE   140514

In [12]:
# select only road way links

print(tm2_link_gdf.CNTYPE.value_counts(dropna = False))

tm2_link_roadway_gdf = tm2_link_gdf[tm2_link_gdf.CNTYPE.isin(["BIKE", "PED", "TANA"])].copy()
print('\n out of {} links in non-Marin legacy TM2 network, {} are roadway links'.format(
    tm2_link_gdf.shape[0],
    tm2_link_roadway_gdf.shape[0]))

# double check with AB node pairs
print('# of unique AB node pairs: {}'.format(tm2_link_roadway_gdf.groupby(["A", "B"]).count().shape[0]))

TANA     807224
MAZ      252437
PED      221905
TAZ       63182
TAP       43512
USE        8981
BIKE       6678
LRAIL       864
CRAIL       144
HRAIL        90
FERRY        52
EXT          44
NaN          32
Name: CNTYPE, dtype: int64

 out of 1405145 links in non-Marin legacy TM2 network, 1035807 are roadway links
# of unique AB node pairs: 1035807


In [82]:
# # Partition TM2 Non Marin for shst Match

# for i in range(14):
#     boundary_gdf = gpd.read_file("../../data/external/county_boundaries/boundary_"+str(i+1)+".geojson")
    
#     sub_gdf = tm2_link_roadway_gdf[tm2_link_roadway_gdf.intersects(boundary_gdf.geometry.unary_union)].copy()
    
#     sub_gdf[["A", "B", "geometry"]].to_file("../../data/external/TM2_nonMarin/tm2nonMarin_"+str(i+1)+".in.geojson",
#                                                 driver = "GeoJSON")

In [13]:
# export raw network data for merging back the conflation result
tm2_link_roadway_gdf.to_file("../../data/external/TM2_nonMarin/tm2nonMarin_raw.geojson",
                             driver = "GeoJSON")

# Prepare TM2 Marin for conflation

In [14]:
%%time
# loading TM2 Marin version data source
# http://bayareametro.github.io/travel-model-two/input/#roadway-network

tm2_marin_link_file = "../../data/external/TM2_Marin/mtc_final_network_base.shp"

# street link layer
tm2_marin_link_gdf = gpd.read_file(tm2_marin_link_file)

# define initial ESPG
# tm2_marin_link_gdf.crs = {"init" : "esri:102646"}
# if the above throws an "invalid crs" error, try the following:
tm2_marin_link_gdf.crs = "esri:102646"

# convert to ESPG lat-lon
tm2_marin_link_gdf = tm2_marin_link_gdf.to_crs({'init': 'epsg:4326'})
print(tm2_marin_link_gdf.crs)

+init=epsg:4326 +type=crs
Wall time: 3min 15s


In [15]:
tm2_marin_link_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1405258 entries, 0 to 1405257
Data columns (total 47 columns):
 #   Column       Non-Null Count    Dtype   
---  ------       --------------    -----   
 0   A            1405258 non-null  int64   
 1   B            1405258 non-null  int64   
 2   F_JNCTID     1405258 non-null  int64   
 3   T_JNCTID     1405258 non-null  int64   
 4   FRC          1405258 non-null  int64   
 5   NAME         741998 non-null   object  
 6   FREEWAY      1405258 non-null  int64   
 7   TOLLRD       291 non-null      object  
 8   ONEWAY       129297 non-null   object  
 9   KPH          1405258 non-null  int64   
 10  MINUTES      1405258 non-null  float64 
 11  CARRIAGE     100 non-null      object  
 12  LANES        1405258 non-null  int64   
 13  RAMP         1405258 non-null  int64   
 14  SPEEDCAT     1405258 non-null  int64   
 15  FEET         1405258 non-null  float64 
 16  ASSIGNABLE   1405258 non-null  int64   
 17  CNTYPE       140520

In [16]:
# select only road way links

print(tm2_marin_link_gdf.CNTYPE.value_counts())

tm2_marin_link_roadway_gdf = tm2_marin_link_gdf[tm2_marin_link_gdf.CNTYPE.isin(["BIKE", "PED", "TANA"])].copy()
print('\n out of {} links in non-Marin legacy TM2 network, {} are roadway links'.format(
    tm2_marin_link_gdf.shape[0],
    tm2_marin_link_roadway_gdf.shape[0]))

# double check with AB node pairs
print('# of unique AB node pairs: {}'.format(tm2_marin_link_roadway_gdf.groupby(["A", "B"]).count().shape[0]))

TANA     807115
MAZ      252435
PED      221905
TAZ       63182
TAP       43894
USE        8797
BIKE       6678
LRAIL       864
CRAIL       144
HRAIL        90
FERRY        52
EXT          44
Name: CNTYPE, dtype: int64

 out of 1405258 links in non-Marin legacy TM2 network, 1035698 are roadway links
# of unique AB node pairs: 1035698


In [17]:
# # Partition TM2 Marin for shst Match 

# for i in range(14):
#     boundary_gdf = gpd.read_file("../../data/external/county_boundaries/boundary_"+str(i+1)+".geojson")
    
#     sub_gdf = tm2_marin_link_roadway_gdf[tm2_marin_link_roadway_gdf.intersects(boundary_gdf.geometry.unary_union)].copy()
    
#     sub_gdf[["A", "B", "geometry"]].to_file("../../data/external/TM2_Marin/tm2Marin_"+str(i+1)+".in.geojson",
#                                                 driver = "GeoJSON")

In [18]:
# export raw network data for merging back the conflation result
tm2_marin_link_roadway_gdf.to_file("../../data/external/TM2_Marin/tm2Marin_raw.geojson",
                                   driver = "GeoJSON")

# Prepared SFCTA true shape file for conflation
Eventually, the conflation result of the SFCTA true shape data was not used.
Please see the Network Rebuild documentation for more details.

In [65]:
# sfcta_file = "../../data/external/stclines/stclines.shp"

# sfcta_gdf = gpd.read_file(sfcta_file)

# sfcta_gdf.crs = {"init" : "epsg:2227"}

# # convert to ESPG lat-lon
# sfcta_gdf = sfcta_gdf.to_crs({'init': 'epsg:4326'})
# sfcta_gdf.crs

# # drop duplicated shapes
# sfcta_gdf.drop_duplicates(subset = sfcta_gdf.drop(["geometry"], axis=1).columns)

# # there's no unique id for sfcta true shape file, not even combinations of A/B
# sfcta_gdf["sfcta_trueshape_id"] = range(1, len(sfcta_gdf) + 1)

# # export for conflation
# sfcta_gdf[["sfcta_trueshape_id", "geometry"]].to_file("../../data/external/sfclines/sfcta.in.geojson",
#                                                 driver = "GeoJSON")

# Prepare SFCTA Stick Network for conflation

In [19]:
sfcta_stick_file = "../../data/external/sfcta/SanFrancisco_links.shp"

sfcta_stick_gdf = gpd.read_file(sfcta_stick_file)

# set initial ESPG
sfcta_stick_gdf.crs = {"init" : "epsg:2227"}

# convert to ESPG lat-lon
sfcta_stick_gdf = sfcta_stick_gdf.to_crs({'init': 'epsg:4326'})
print(sfcta_stick_gdf.crs)

+init=epsg:4326 +type=crs


In [20]:
print('# of links: {}'.format(sfcta_stick_gdf.shape[0]))
print(sfcta_stick_gdf.columns)

# of links: 61875
Index(['A', 'B', 'TOLL', 'USE', 'CAP', 'AT', 'FT', 'STREETNAME', 'TYPE',
       'MTYPE', 'SPEED', 'DISTANCE', 'TIME', 'LANE_AM', 'LANE_OP', 'LANE_PM',
       'BUSLANE_AM', 'BUSLANE_OP', 'BUSLANE_PM', 'TOLLAM_DA', 'TOLLAM_SR2',
       'TOLLAM_SR3', 'TOLLPM_DA', 'TOLLPM_SR2', 'TOLLPM_SR3', 'TOLLEA_DA',
       'TOLLEA_SR2', 'TOLLEA_SR3', 'TOLLMD_DA', 'TOLLMD_SR2', 'TOLLMD_SR3',
       'TOLLEV_DA', 'TOLLEV_SR2', 'TOLLEV_SR3', 'VALUETOLL_F', 'PASSTHRU',
       'BUSTPS_AM', 'BUSTPS_OP', 'BUSTPS_PM', 'TSVA', 'TSIN', 'BIKE_CLASS',
       'PER_RISE', 'ONEWAY', 'PROJ', 'DTA_EDIT_FL', 'TOLLTIME', 'PHASE',
       'ACTION', 'geometry'],
      dtype='object')


In [22]:
# only conflation SF part of the network
boundary_4_gdf = gpd.read_file("../../data/external/county_boundaries/boundary_4.geojson")
sfcta_stick_gdf = sfcta_stick_gdf[sfcta_stick_gdf.intersects(boundary_4_gdf.geometry.unary_union)]

# remove "special facility"
sfcta_stick_roadway_gdf = sfcta_stick_gdf[~(sfcta_stick_gdf.FT == 6)].copy()

In [23]:
# Write out SFCTA stick network for conflation

sfcta_stick_roadway_gdf[['A', 'B', "geometry"]].to_file("../../data/external/sfcta/sfcta_in.geojson",
                                                        driver="GeoJSON")

In [24]:
sfcta_stick_roadway_gdf.to_file("../../data/external/sfcta/sfcta_raw.geojson",
                                driver="GeoJSON")

# Prepare PEMS for conflation

In [27]:
pems_file = "../../data/external/mtc/pems_period.csv"
pems_df = pd.read_csv(pems_file)
print(pems_df.columns)

Index(['station', 'district', 'route', 'direction', 'type', 'time_period',
       'lanes', 'median_flow', 'avg_flow', 'sd_flow', 'median_speed',
       'avg_speed', 'sd_speed', 'median_occup', 'avg_occup', 'sd_occupancy',
       'days_observed', 'state_pm', 'abs_pm', 'latitude', 'longitude', 'year'],
      dtype='object')


In [28]:
# create geometry from X and Y
pems_df["geometry"] = [Point(xy) for xy in zip(pems_df.longitude, pems_df.latitude)]

pems_gdf = gpd.GeoDataFrame(pems_df, geometry = pems_df["geometry"],
                            crs={"init" : "epsg:4326"})

# drop records missing geometry
pems_gdf = pems_gdf[~((pems_gdf.longitude.isnull()) | (pems_gdf.latitude.isnull()))]

In [29]:
# export for conflation
pems_gdf.drop_duplicates(subset = ["station", "longitude", "latitude"],
                         inplace=True)[["station", "longitude", "latitude", "geometry"]].to_file(
    "../../data/external/mtc/pems.in.geojson", driver = "GeoJSON")

In [30]:
# export modified pems_gdf
pems_gdf.to_file("../../data/external/mtc/pems_raw.geojson", driver = "GeoJSON")